# EvapoTranspiration Using Pennmans Formula on - Gridded Data EMDA
## Ensemble of Meterological Dataset for North America 
### using probabilistc methods to estimate the uncertainty in spatial fields
https://essd.copernicus.org/articles/13/3337/2021/

*Each scenario represents a predicted socio-economic status and the radiative energy that the globe will reach by the end of the century (4 socio-economic scenarios paired with radiative force we are producing (KJ/m2)).  **SSP1-2.6** = assumes reduced emissions, **SSP2-4.5** = assumes the same trend of emissions as historical, **SSP3-7.0** = medium to high emissions scenario, **SSP5-8.5**=optimal for economic development but with high emissions.*

![Penman Formula](penman.jpg)


# Install Libraries

In [174]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from scipy.stats import linregress
from itertools import combinations
import math
from matplotlib.dates import MonthLocator, DateFormatter
import requests
from io import StringIO
import os

# Hard Coded Variables

## Observed

In [175]:
observed_folder_t_p = 'C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/TM_PHES_code/Observed'
observed_folder_trange = 'C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/TM_PHES_code/Observed/trange'

## S1

In [176]:
s1_folder_t_p = 'C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/TM_PHES_code/S1'
s1_folder_trange = 'C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/TM_PHES_code/S1/trange'

## S2

In [177]:
s2_folder_t_p = 'C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/TM_PHES_code/S2'
s2_folder_trange = 'C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/TM_PHES_code/S2/trange'

## S3

In [178]:
s3_folder_t_p = 'C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/TM_PHES_code/S3'
s3_folder_trange = 'C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/TM_PHES_code/S3/trange'

## S4

In [179]:
s4_folder_t_p = 'C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/TM_PHES_code/S4'
s4_folder_trange = 'C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/TM_PHES_code/S4/trange'

# Utility Functions

In [180]:
def read_csv_files_in_folder(folder_path):
    dataframes = []
    
    files = sorted(os.listdir(folder_path))  # Sort files alphabetically
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(folder_path, file)
            df = pd.read_csv(file_path)
            dataframes.append(df)
    
    return dataframes


In [181]:
def merge_dataframes(df_t_p_list, df_trange_list):
    """
    Merge two lists of DataFrames based on the 'Date' column.

    Parameters:
    - df_t_p_list: List of DataFrames with 'Date', 'tmean', and 'precip' columns, sorted by 'Date'
    - df_trange_list: List of DataFrames with 'Date' and 'trange' columns, sorted by 'Date'

    Returns:
    - Merged list of DataFrames on 'Date' column
    """
    merged_dfs = []

    # Ensure the length of both lists is the same
    assert len(df_t_p_list) == len(df_trange_list), "The lengths of input lists must be the same."

    for df_t_p, df_range in zip(df_t_p_list, df_trange_list):
        # Convert 'Date' column to datetime if not already
        if not pd.api.types.is_datetime64_any_dtype(df_t_p['Date']):
            df_t_p['Date'] = pd.to_datetime(df_t_p['Date'])

        if not pd.api.types.is_datetime64_any_dtype(df_range['Date']):
            df_range['Date'] = pd.to_datetime(df_range['Date'])

        merged_df = pd.merge(df_t_p, df_range, on='Date', how='inner') 
        merged_df = merged_df.sort_values(by='Date')
        merged_dfs.append(merged_df)

    return merged_dfs

In [182]:
def calculate_and_add_column(df_list):
    """
    Calculate a new column based on the specified formula and add it to each DataFrame in the list.

    Parameters:
    - df_list: List of DataFrames

    Returns:
    - None (modifies the DataFrames in place)
    """
    for df in df_list:
        # Extract year, month, and day information
        df['Year'] = df['Date'].dt.year
        df['Month'] = df['Date'].dt.month
        df['Day'] = df['Date'].dt.day

        elev = 1426  # elevation meters
        lat = 49.575 # latitude deg

        # Calculate tmean_tdew
        df['rann'] = df.groupby('Month')['trange'].transform('mean')
        df['tmean_tdew'] = (
            0.0023 * elev + 0.37 * df['tmean'] + 0.53 * df['trange'] + 0.35 * df['rann'] - 10.9
        )

        # Calculate Evaporation

        df['evaporation'] = (
            (524 * (df['tmean'] + 0.006 * elev) / (100 - lat) + 15 * df['tmean_tdew']) / (80 - df['tmean'])
        ).where(df['tmean'] >= 0, other=0)  # Applying condition for tmean < 0

        # Drop the intermediate columns
        df.drop(['Year', 'Month', 'Day', 'rann', 'tmean_tdew'], axis=1, inplace=True, errors='ignore') 

In [183]:
def fill_missing_dates(dataframes_list):
    """
    Fill missing dates in each DataFrame in the given list using linear interpolation.

    Parameters:
    - dataframes_list: List of DataFrames

    Returns:
    - List of DataFrames with missing dates filled using linear interpolation
    """
    filled_dataframes = []

    for df in dataframes_list:
        if 'Date' in df.columns and not pd.api.types.is_datetime64_any_dtype(df['Date']):
            df['Date'] = pd.to_datetime(df['Date'])
            
        if not df.empty:  # Check if DataFrame contains data, some dataframes are empty, some realizations were given without data, dropping those here
            start_date = min(df['Date'])
            end_date = max(df['Date'])
            date_range = pd.date_range(start=start_date, end=end_date)
            full_dates_df = pd.DataFrame({'Date': date_range})
            merged_df = pd.merge(full_dates_df, df, on='Date', how='left')
            merged_df = merged_df.sort_values(by='Date')
            filled_df = merged_df.interpolate(method='linear')
            filled_dataframes.append(filled_df)


    return filled_dataframes

In [184]:
def count_nan_values(dataframes_list):
    """
    Count the number of NaN values for each DataFrame in the given list.

    Parameters:
    - dataframes_list: List of DataFrames

    Returns:
    - List containing count of NaN values for each DataFrame
    """
    nan_counts = []
    
    for df in dataframes_list:
        nan_counts.append(df.isna().sum().sum())

    return nan_counts

In [185]:
def detect_missing_dates_in_list(dataframes_list):
    """
    Detect missing dates in the 'Date' column of each DataFrame in the given list.

    Parameters:
    - dataframes_list: List of DataFrames with a 'Date' column

    Returns:
    - List of booleans indicating if there are missing dates in each DataFrame
    """
    missing_dates_list = []

    for df in dataframes_list:
        # Ensure 'Date' column is datetime type
        if 'Date' in df.columns and not pd.api.types.is_datetime64_any_dtype(df['Date']):
            df['Date'] = pd.to_datetime(df['Date'])

        # Get the expected date range
        min_date = df['Date'].min()
        max_date = df['Date'].max()
        expected_dates = pd.date_range(start=min_date, end=max_date)

        # Check for missing dates
        missing_dates = ~df['Date'].isin(expected_dates)
        missing_dates_list.append(missing_dates.any())

    return missing_dates_list


# Data Engineering

## Observed

In [186]:
observed_t_p = read_csv_files_in_folder(observed_folder_t_p)
observed_trange = read_csv_files_in_folder(observed_folder_trange)

In [187]:
observed_dfs = merge_dataframes(observed_t_p, observed_trange)

In [188]:
observed_dfs[0].head(2)

,Date,tmean,precip,trange
0,1979-01-01,-24.9175,2.77110,12.577
1,1979-01-02,-21.8485,0.70797,9.769


## S1

In [189]:
s1_t_p = read_csv_files_in_folder(s1_folder_t_p)
s1_trange = read_csv_files_in_folder(s1_folder_trange)

In [190]:
s1_dfs = merge_dataframes(s1_t_p, s1_trange)

## S2

In [191]:
s2_t_p = read_csv_files_in_folder(s2_folder_t_p)
s2_trange = read_csv_files_in_folder(s2_folder_trange)

In [192]:
s2_dfs = merge_dataframes(s2_t_p, s2_trange)

## S3

In [193]:
s3_t_p = read_csv_files_in_folder(s3_folder_t_p)
s3_trange = read_csv_files_in_folder(s3_folder_trange)

In [194]:
s3_dfs = merge_dataframes(s3_t_p, s3_trange)

## S4

In [195]:
s4_t_p = read_csv_files_in_folder(s4_folder_t_p)
s4_trange = read_csv_files_in_folder(s4_folder_trange)

In [196]:
s4_dfs = merge_dataframes(s4_t_p, s4_trange)

In [197]:
s4_dfs[39].head()

,Date,precip,tmean,trange
0,2015-01-01,6.514761,1.966477,4.000172
1,2015-01-02,8.052436,2.692204,4.202973
2,2015-01-03,1.416353,0.828085,3.932778
3,2015-01-04,23.150974,2.364892,5.937545
4,2015-01-05,5.724569,-1.390870,6.359421


# Calculations

## Observed

In [198]:
calculate_and_add_column(observed_dfs)

In [199]:
observed_dfs[0].head()

,Date,tmean,precip,trange,evaporation
0,1979-01-01,-24.9175,2.771100,12.577,0.0
1,1979-01-02,-21.8485,0.707970,9.769,0.0
2,1979-01-03,-22.2550,0.117580,9.818,0.0
3,1979-01-04,-23.5575,0.000000,12.729,0.0
4,1979-01-05,-21.4305,0.000486,13.063,0.0


In [200]:
filled_observed_dfs = fill_missing_dates(observed_dfs)

In [201]:
# distinct_values = filled_observed_dfs[0]['rann'].unique()
# print(distinct_values)

In [202]:
# mean_evaporation_by_month = filled_observed_dfs[0].groupby('Month')['evaporation'].mean()
# print(mean_evaporation_by_month)

In [203]:
# days_in_month = filled_observed_dfs[0].groupby('Month')['Day'].nunique()

# mean_evaporation_by_month = filled_observed_dfs[0].groupby('Month')['evaporation'].mean()

# total_evaporation_by_month = mean_evaporation_by_month * days_in_month

# print(total_evaporation_by_month)

In [204]:
# total_evaporation_sum = total_evaporation_by_month.sum()

# print("Total evaporation:", total_evaporation_sum)

In [205]:
# Iterate over the merged dataframes for saving
for i, df in enumerate(filled_observed_dfs):
    # Save the merged dataframe to a CSV file
    csv_filename = f"observed_t_p_e.csv"  # Naming based on position in the list
    csv_filepath = os.path.join("C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/TM_PHES_code/observed/penman", csv_filename)
    df.to_csv(csv_filepath, index=False)


## **Scenario 1:** SSP1-2.6 = *assumes reduced emissions*

In [206]:
calculate_and_add_column(s1_dfs)

In [207]:
s1_dfs[0].head()

,Date,precip,tmean,trange,evaporation
0,2015-01-01,12.284951,-4.077306,2.030853,0.0
1,2015-01-02,2.865495,-3.112623,0.101487,0.0
2,2015-01-03,6.831872,-0.506162,5.111435,0.0
3,2015-01-04,1.807658,-5.040199,10.397952,0.0
4,2015-01-05,1.872311,-3.142410,4.184681,0.0


In [208]:
filled_s1_dfs = fill_missing_dates(s1_dfs)

In [209]:
# Iterate over the merged dataframes for saving
for i, df in enumerate(filled_s1_dfs):
    # Save the merged dataframe to a CSV file
    csv_filename = f"s1_{i+1}_t_p_e.csv"  # Naming based on position in the list
    csv_filepath = os.path.join("C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/TM_PHES_code/S1/penman", csv_filename)
    df.to_csv(csv_filepath, index=False)


## **Scenario 2:** SSP2-4.5 = *assumes the same trend of emissions as historical*

In [210]:
calculate_and_add_column(s2_dfs)

In [211]:
s2_dfs[0].head()

,Date,precip,tmean,trange,evaporation
0,2015-01-01,5.913979,1.064339,3.946321,0.838265
1,2015-01-02,7.351886,1.542983,4.307355,0.977218
2,2015-01-03,1.266364,-0.072935,3.493105,0.000000
3,2015-01-04,19.761959,1.357572,6.058133,1.114316
4,2015-01-05,5.841811,-3.309855,7.554837,0.000000


In [212]:
filled_s2_dfs = fill_missing_dates(s2_dfs)

In [213]:
# Iterate over the merged dataframes for saving
for i, df in enumerate(filled_s2_dfs):
    # Save the merged dataframe to a CSV file
    csv_filename = f"s2_{i+1}_t_p_e.csv"  # Naming based on position in the list
    csv_filepath = os.path.join("C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/TM_PHES_code/S2/penman", csv_filename)
    df.to_csv(csv_filepath, index=False)


## **Scenario 3:** SSP3-7.0 = *medium to high emissions scenario*

In [214]:
calculate_and_add_column(s3_dfs)

In [215]:
s3_dfs[5].tail()

,Date,precip,tmean,trange,evaporation
31385,2100-12-06,5.592901,-0.142797,1.160828,0.000000
31386,2100-12-07,0.000000,0.141497,2.574369,0.422546
31387,2100-12-08,0.000000,-0.309825,2.516676,0.000000
31388,2100-12-09,0.000000,-7.302217,6.388819,0.000000
31389,2100-12-10,0.895917,-13.565012,5.709206,0.000000


In [216]:
filled_s3_dfs = fill_missing_dates(s3_dfs)

In [217]:
filled_s3_dfs[5].tail()

,Date,precip,tmean,trange,evaporation
31385,2100-12-06,5.592901,-0.142797,1.160828,0.000000
31386,2100-12-07,0.000000,0.141497,2.574369,0.422546
31387,2100-12-08,0.000000,-0.309825,2.516676,0.000000
31388,2100-12-09,0.000000,-7.302217,6.388819,0.000000
31389,2100-12-10,0.895917,-13.565012,5.709206,0.000000


In [218]:
# Iterate over the merged dataframes for saving
for i, df in enumerate(filled_s3_dfs):
    # Save the merged dataframe to a CSV file
    csv_filename = f"s3_{i+1}_t_p_e.csv"  # Naming based on position in the list
    csv_filepath = os.path.join("C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/TM_PHES_code/S3/penman", csv_filename)
    df.to_csv(csv_filepath, index=False)


## **Scenario 4:** SSP5-8.5 = *optimal for economic development but with high emissions.*

In [219]:
calculate_and_add_column(s4_dfs)

In [220]:
s4_dfs[0].head()

,Date,precip,tmean,trange,evaporation
0,2015-01-01,6.129145,1.677566,3.966279,0.925677
1,2015-01-02,4.294818,-0.198543,5.185653,0.000000
2,2015-01-03,0.000000,-1.132881,4.745145,0.000000
3,2015-01-04,1.047620,-2.067218,4.304637,0.000000
4,2015-01-05,1.139335,-2.429765,5.007710,0.000000


In [221]:
s4_dfs[39].head()

,Date,precip,tmean,trange,evaporation
0,2015-01-01,6.514761,1.966477,4.000172,1.002543
1,2015-01-02,8.052436,2.692204,4.202973,1.182462
2,2015-01-03,1.416353,0.828085,3.932778,0.752140
3,2015-01-04,23.150974,2.364892,5.937545,1.287890
4,2015-01-05,5.724569,-1.390870,6.359421,0.000000


In [222]:
filled_s4_dfs = fill_missing_dates(s4_dfs)

In [223]:
# distinct_values = filled_s4_dfs[0]['rann'].unique()
# print(distinct_values)

In [224]:
# days_in_month = filled_s4_dfs[0].groupby('Month')['Day'].nunique()

# mean_evaporation_by_month = filled_s4_dfs[0].groupby('Month')['evaporation'].mean()

# total_evaporation_by_month = mean_evaporation_by_month * days_in_month

# print(total_evaporation_by_month)

In [225]:
# total_evaporation_sum = total_evaporation_by_month.sum()

# print("Total evaporation:", total_evaporation_sum)

In [226]:
# Iterate over the merged dataframes for saving
for i, df in enumerate(filled_s4_dfs):
    # Save the merged dataframe to a CSV file
    csv_filename = f"s4_{i+1}_t_p_e.csv"  # Naming based on position in the list
    csv_filepath = os.path.join("C:/Users/14037/OneDrive - University of Calgary/Documents/ENCI_570/TM_PHES_code/S4/penman", csv_filename)
    df.to_csv(csv_filepath, index=False)


# HEC HMS Quality Check

## No nan values

In [227]:
nan_counts = count_nan_values(filled_observed_dfs)

print(nan_counts)

[0]


In [228]:
nan_counts = count_nan_values(filled_s1_dfs)

print(nan_counts)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [229]:
nan_counts = count_nan_values(filled_s2_dfs)

print(nan_counts)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [230]:
nan_counts = count_nan_values(filled_s3_dfs)

print(nan_counts)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [231]:
nan_counts = count_nan_values(filled_s4_dfs)

print(nan_counts)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## No missing dates

In [232]:
missing_dates_result = detect_missing_dates_in_list(filled_observed_dfs)
print("Missing Dates:", missing_dates_result)

Missing Dates: [False]


In [233]:
missing_dates_result = detect_missing_dates_in_list(filled_s1_dfs)
print("Missing Dates:", missing_dates_result)

Missing Dates: [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]


In [234]:
missing_dates_result = detect_missing_dates_in_list(filled_s2_dfs)
print("Missing Dates:", missing_dates_result)

Missing Dates: [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]


In [235]:
missing_dates_result = detect_missing_dates_in_list(filled_s3_dfs)
print("Missing Dates:", missing_dates_result)

Missing Dates: [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]


In [236]:
missing_dates_result = detect_missing_dates_in_list(filled_s4_dfs)
print("Missing Dates:", missing_dates_result)

Missing Dates: [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
